In [ ]:
import os
# Imports the OS module, which handles file paths and directory navigation across different operating systems.
import csv
# Imports the CSV module, which allows us to format and save our results directly into a spreadsheet format.
import random
# Imports the random module, which gives us the tools to randomly select our satellite bands.
import subprocess
# Imports subprocess, which allows this script to type commands into the terminal and run other scripts (like evaluate.py).
import re
# Imports the regular expressions (re) module, which is a search tool we'll use to extract the exact MAE number from the terminal output.

# --- CONFIGURATION VARIABLES ---
IMAGERY_PATH = "/path/to/your/imagery"
# Defines the location of the KidSat data. YOU MUST CHANGE THIS to the actual path on your supercomputer!
NUM_TESTS = 10
# Sets the maximum number of unique random band combinations to test so the supercomputer doesn't run forever.
AVAILABLE_BANDS = list(range(1, 14))
# Creates a list of available bands from 1 to 13 (Sentinel-2 imagery uses 13 bands).
CSV_FILENAME = "random_bands_mae_results.csv"
# Sets the name of the file where our MAE scores will be saved in the same folder the script is run from.

def get_random_bands(num_bands=3):
# Defines a function to pick our band combinations, defaulting to 3 bands.
    selected_bands = random.sample(AVAILABLE_BANDS, num_bands)
    # Uses random.sample to pick 3 bands without replacement, completely preventing duplicate bands like [1, 1, 3].
    selected_bands.sort()
    # Sorts the bands numerically (so [3, 2, 1] becomes [1, 2, 3]), ensuring we don't test the same combination twice.
    return tuple(selected_bands)
    # Returns the sorted selection as an immutable 'tuple', which allows us to track it in our history.

def extract_mae(terminal_output):
# Defines a function to scan the text that `evaluate.py` prints out and find the MAE score.
    match = re.search(r'MAE:\s*([0-9]+\.[0-9]+)', terminal_output, re.IGNORECASE)
    # Uses regex to look for "MAE:" followed by numbers and a decimal. NOTE: You may need to tweak "MAE:" if their script prints it differently (e.g., "Mean Absolute Error:").
    if match:
    # Checks if the search successfully found a matching number.
        return float(match.group(1))
        # Extracts the number, converts it to a float (a decimal format), and returns it to our loop.
    return None
    # Returns 'None' if the script crashed or the MAE wasn't printed.

if __name__ == "__main__":
# A safety check ensuring this code only runs if you execute this specific file directly.
    tested_combos = set()
    # Creates an empty 'set' (a mathematical list with no duplicates) to log the combinations we've already tried.
    
    with open(CSV_FILENAME, mode='w', newline='') as csv_file:
    # Opens our CSV file in 'write' mode, ensuring no weird blank rows are added between our data entries.
        writer = csv.writer(csv_file)
        # Initializes the CSV writer tool to format our text into columns.
        writer.writerow(['Band_1', 'Band_2', 'Band_3', 'Fold_1', 'Fold_2', 'Fold_3', 'Fold_4', 'Fold_5', 'Avg_MAE'])
        # Writes the top header row of our spreadsheet so the columns are properly labeled.
        
        while len(tested_combos) < NUM_TESTS:
        # Starts a loop that will keep running until we successfully test the desired number of unique combinations.
            combo = get_random_bands()
            # Calls our function to generate a new 3-band combination.
            
            if combo in tested_combos:
            # Checks if this exact combination has already been tested.
                continue
                # If it has, we 'continue' (skip the rest of the loop) and draw a new random combination instantly.
            
            tested_combos.add(combo)
            # Adds this brand-new combination to our tracking set so we don't accidentally do it again later.
            print(f"Testing Bands: {combo}")
            # Prints a message to your supercomputer console so you know where the script is currently at.
            
            fold_maes = []
            # Creates an empty list to collect the 5 MAE scores (one for each folder/fold) for this specific band combination.
            
            for fold in range(1, 6):
            # Starts an inner loop that counts from 1 to 5, representing your cross-validation folders.
                
                command = [
                # Builds the exact command-line instruction we want to send to the terminal.
                    "python", "modelling/dino/evaluate.py",
                    # Tells Python to run the repository's existing evaluation script.
                    "--imagery_path", IMAGERY_PATH,
                    # Passes the location of your KidSat dataset.
                    "--imagery_source", "S",
                    # Explicitly flags the data as Sentinel imagery (S), matching the original author's parameters.
                    "--mode", "spatial",
                    # Sets the evaluation mode to 'spatial' as required by their setup.
                    "--fold", str(fold),
                    # Passes the current folder/fold number (1 through 5) to the evaluation script.
                    "--grouped_bands", str(combo[0]), str(combo[1]), str(combo[2])
                    # Passes our 3 randomly selected bands as the final arguments.
                ]
                
                print(f"  Running Fold {fold}...")
                # Prints a quick status update to the console.
                
                result = subprocess.run(command, capture_output=True, text=True)
                # Executes the command we built, captures everything the script tries to print, and saves it as text.
                
                mae = extract_mae(result.stdout)
                # Runs our extraction function to locate the MAE score inside the captured text.
                
                if mae is None:
                # Checks if the extraction failed, which usually means the underlying `evaluate.py` script crashed.
                    print(f"  Error in Fold {fold}. Details: {result.stderr[:200]}")
                    # Prints the first 200 characters of the error log so you can debug what went wrong.
                    mae = "Error"
                    # Records the word "Error" instead of a number so our CSV columns stay perfectly aligned.
                
                fold_maes.append(mae)
                # Appends the fold's MAE score (or the error warning) to our temporary list.
            
            valid_maes = [m for m in fold_maes if isinstance(m, float)]
            # Uses a list comprehension to filter out any "Error" words, leaving only the successful decimal numbers.
            
            if valid_maes:
            # Checks to make sure at least one of the 5 folds completed successfully.
                avg_mae = sum(valid_maes) / len(valid_maes)
                # Calculates the average MAE across the successful folds.
            else:
            # If the list of valid numbers is empty (all 5 folds crashed).
                avg_mae = "Error"
                # Sets the overall average to "Error".
                
            writer.writerow([combo[0], combo[1], combo[2]] + fold_maes + [avg_mae])
            # Writes the three bands, the 5 individual fold scores, and the final average score into a new row in the CSV.
            
            csv_file.flush()
            # Forces the supercomputer to save the CSV data to the disk immediately (prevents data loss if the job hits a time limit!).
            
            print(f"Finished {combo}. Avg MAE: {avg_mae}\n")
            # Prints the final average score for the combination before looping back to start the next one.

100%|██████████| 4/4 [00:02<00:00,  1.54it/s]


start
Test score (MAE): 0.1554161289806102


100%|██████████| 4/4 [00:04<00:00,  1.06s/it]


start
Test score (MAE): 0.16068112561225867


100%|██████████| 4/4 [00:04<00:00,  1.14s/it]


start
Test score (MAE): 0.16200557977329602


100%|██████████| 4/4 [00:04<00:00,  1.15s/it]


start
Test score (MAE): 0.16801169055165058


100%|██████████| 4/4 [00:04<00:00,  1.13s/it]


start
Test score (MAE): 0.15644389039711698


In [18]:
np.mean(test_scores), np.std(test_scores)/np.sqrt(5)

(0.16051168306298647, 0.002010195603094569)